# DayTime Checkout

This notebook is intended to run a series of tests to ensure the system responds as expected.
It begins with LATISS, but can also be used with the telescope ATCS components only.

In [ ]:
import asyncio
import time
import os
import numpy as np
import logging

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

In [ ]:
logger.info(os.environ["OSPL_URI"])
logger.info(os.environ["LSST_DDS_PARTITION_PREFIX"])

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

## Indicate if LATISS is part of the checkout

In [ ]:
with_latiss=True

## Enable LATISS

In [ ]:
if with_latiss:
    await latiss.enable()

If the camera and/or archiver don't come up, see section `Quick Troubleshooting` for support commands. 

## LATISS image and ingestion verification

### Bias verification

In [ ]:
if with_latiss:
    latiss.rem.atoods.evt_imageInOODS.flush()
    await latiss.take_bias(nbias=1)
    ingest_event = await latiss.rem.atoods.evt_imageInOODS.next(flush=False, timeout=10)
    logging.info(ingest_event)
    assert ingest_event.statusCode == 0, 'Ingestion was not successful!'

Check that the bias pops up in the monitor (https://roundtable.lsst.codes/rubintv/monitor_current)
If it fails, let Patrick or Merlin know

You can check that it ingested at NCSA with this link:
This might take a few minutes.

https://lsst-sandbox.ncsa.illinois.edu/transfer-ingest-monitor/

### Engineering test frame verification
This is analogous to a science frame

In [ ]:
if with_latiss:
    latiss.rem.atoods.evt_imageInOODS.flush()
    await latiss.take_engtest(2, filter='SDSSg',grating=3)
    ingest_event = await latiss.rem.atoods.evt_imageInOODS.next(flush=False, timeout=10)
    logging.info(ingest_event)
    assert ingest_event.statusCode == 0, 'Ingestion was not successful!'

## Enable ATCS

In [ ]:
await atcs.enable()

If any of the ATCS CSCs failed to transition to enable state with the above command, see `Quick Troubleshooting` section for more commands. 

## Pneumatics Functionality Verification

Check ATPneumatics air pressure. 

In [ ]:
await atcs.open_valves()
pressure = await atcs.rem.atpneumatics.tel_mainAirSourcePressure.next(flush=True, timeout=5)
if pressure.pressure > 300000:
    logging.info(f'Air pressure is {pressure.pressure:0.0f} Pascals, which is fine.')
else:
    raise AssertionError(f'Air pressure is {pressure.pressure}, which is too low. It needs to be between ~275790 and ~413000 Pascals (40 and 60 PSI). \n Check that compressor and dryer is running. Then check that the regulator inside the pier is set correctly. ')

Turn on ATAOS correction(s), without spectrograph (if the tests were skipped above).<br>
Note there is a race condition in the ATPneumatics that might result in this failing, so you'll have to run in twice

In [ ]:
await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)

In [ ]:
pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.aget(timeout=5)
logging.info(pressure)

Turn off ATAOS correction(s), without spectrograph (if the tests were skipped above)

In [ ]:
await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)

Lower mirror back on hardpoints

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=0)

Turn on ATAOS correction(s), without spectrograph (if the tests were skipped above). <br>
This should not ever fail.

In [ ]:
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)

## Start Telescope testing

Turn off dome following

In [ ]:
await atcs.disable_dome_following()

### Test Point AzEl

In [ ]:
start_az=0.0
start_el=80.0
start_rot=0
await atcs.point_azel(az=start_az, el=start_el, rot_tel=start_rot)

In [ ]:
# Stop tracking
tmp=await atcs.stop_tracking()
logging.info(tmp)

### Test Sidereal tracking
but do so starting from the same position

In [ ]:
coord=atcs.radec_from_azel(az=start_az+5, el=start_el-5)
await atcs.slew_icrs(coord.ra, coord.dec, rot=start_rot, stop_before_slew=False)

In [ ]:
# Stop tracking
await atcs.stop_tracking()

## Verify dome functions

In [ ]:
# Check that dome moves
dome_az = await atcs.rem.atdome.tel_position.next(flush=True,timeout=10)
logging.info(f'Dome currently thinks it is at an azimuth position of {dome_az.azimuthPosition}.\n Note the dome may not be properly homed at this time')
d_az=15
await atcs.rem.atdome.cmd_moveAzimuth.set_start(azimuth=dome_az.azimuthPosition+d_az)

Setup telescope to run the "prepare for flats" script so it maximizes the internal functionality of the script

In [ ]:
# Turn off ATAOS correction(s)
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)
# put mirror back on the hardpoints
pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.aget(timeout=5)
logging.info(pressure)

In [ ]:
# shut off the valves so we can test the pneumatics
await atcs.rem.atpneumatics.cmd_closeMasterAirSupply.start()
await atcs.rem.atpneumatics.cmd_closeInstrumentAirValve.start()

In [ ]:
# Open mirror covers and vents
# Note that there is currently a race conditions that might make this fail in the first attempt
await atcs.open_m1_cover()
await atcs.open_m1_vent()

In [ ]:
# Close mirror covers and vents
await atcs.close_m1_cover()
await atcs.close_m1_vent()

## Run prepare_for_flats
First we home the dome to make sure it knows where it is.

In [ ]:
await atcs.home_dome()

In [ ]:
await atcs.prepare_for_flatfield()

## Perform slew of Full Observatory
This will also take test images

Now close the mirror cover just to keep things safe - this will move the telescope to a higher position (El~70)

In [ ]:
await atcs.close_m1_cover()
await atcs.close_m1_vent()

Turn on dome following which will align the dome with the telescope

In [ ]:
await atcs.enable_dome_following()

Turn on ATAOS correction(s), without spectrograph (if the tests were skipped above). <br>
Note there is a race condition in the ATPneumatics that might result in this failing. <br>
If it fails, you'll have to run in twice

In [ ]:
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)

Now start tracking a siderial target, but starting from the same position. <br>
Then take a test image engineering test frame (analogous to a science frame) and make sure it lands

In [ ]:
start_az=10
start_el=75
start_rot=0
coord=atcs.radec_from_azel(az=start_az, el=start_el)
await atcs.slew_icrs(coord.ra, coord.dec, rot=start_rot, stop_before_slew=False)

In [ ]:
if with_latiss:
    latiss.rem.atoods.evt_imageInOODS.flush()
    await latiss.take_engtest(2, filter=3, grating=3)
    ingest_event = await latiss.rem.atoods.evt_imageInOODS.next(flush=False, timeout=10)
    logging.info(ingest_event)
    assert ingest_event.statusCode == 0, 'Ingestion was not successful!'

Pause here and let it track for at least a couple minutes

### Slew to target #2
Then take an image with a different spectrograph setup.

In [ ]:
start_az=0.0
start_el=80
start_rot=0
coord=atcs.radec_from_azel(az=start_az, el=start_el)
await atcs.slew_icrs(coord.ra, coord.dec, rot=start_rot, stop_before_slew=False)

In [ ]:
if with_latiss:
    latiss.rem.atoods.evt_imageInOODS.flush()
    await latiss.take_engtest(2, filter=0, grating=0)
    ingest_event = await latiss.rem.atoods.evt_imageInOODS.next(flush=False, timeout=10)
    logging.info(ingest_event)
    assert ingest_event.statusCode == 0, 'Ingestion was not successful!'

## Shutdown all ATCS components

In [ ]:
await atcs.shutdown()

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)

## Put LATISS in standby

In [ ]:
await latiss.standby()

# Quick Troubleshooting

## LATISS CSCs failed to transition to ENABLED state. 
The current DDS configuration results in history being lost when CSCs are restarted and/or a small network outage occurs. If this happens, then to receive the summaryState event, which is required by salobj.get_summary_state (and used extensively in the atcs), it is necessary to manually change the state of a CSC. The commands below can be used to perform this action.

A new DDS configuration is now ready to be tested and will hopefully be released in the near future.

In [ ]:
# ATOODs didn't come up

In [ ]:
tmp=await latiss.rem.atoods.cmd_start.set_start()
print(tmp)

In [ ]:
await salobj.set_summary_state(latiss.rem.atoods, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(latiss.rem.atoods, salobj.State.ENABLED)

In [ ]:
# ATCamera didn't come up

In [ ]:
tmp=await latiss.rem.atcamera.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.ENABLED)

## ATCS CSCs failed to transition to ENABLED state. 

In [ ]:
# ATHexapod 

In [ ]:
tmp=await atcs.rem.athexapod.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.ENABLED)

In [ ]:
# ATPneumatics

In [ ]:
tmp=await atcs.rem.atpneumatics.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.ENABLED)

In [ ]:
# ATPtg 

In [ ]:
tmp=await atcs.rem.atptg.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)